<a href="https://colab.research.google.com/github/JuCarv-bit/exercises-deep-learning-curriculum/blob/main/Notebook_running_shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Theoretical Discussion - Week 1

**- What is different architecturally from the Transformer, vs a normal RNN, like an LSTM? (Specifically, how are recurrence and time managed?)**

 Transformers work with parallel inputs (multi-head attention), while RNNs and LSTMs have a time component and a hidden state. Recurrent layers are present in RNN, while transform uses MHA layers.





- Attention is defined as, Attention(Q,K,V) = softmax(QK^T/sqrt(d_k))V. What are the dimensions for Q, K, and V? Why do we use this setup? What other combinations could we do with (Q,K) that also output weights?

The dimension of the matrix Query $Q$ and $K$ is $(dk, d_{model})$, and Value is $(dv, d_{model})$.

Computation of multiheaded attention (mostly) constant.

The dimensions of (Q,K) must be equal



- Are the dense layers different at each multi-head attention block? Why or why not?

The architecture is the same (self-attention + feed forward layer)


- Why do we have so many skip connections, especially connecting the input of an attention function to the output? Intuitively, what if we didn't?

The skipping connections prevent the vanishing and exploding gradient. The model can have a worst performance if not using it.

In [1]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

In [2]:
vocab_size = 10
d_model = 512 #128

num_heads = 8 #4
num_layers = 6 #5
dropout = 0.1 #0.3
batch_size = 32 #
len_seq = 6

# 5000 permutacoes de 8 digitos

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
seq = [2, 8, 8, 0, 7, 1, 1]

# X = torch.tensor(seq).unsqueeze(0).to(torch.int64).to(device)
# y = torch.flip(X, (-1,)).to(torch.int64).to(device)

class GenerateDataset():
    def __init__(self, numdigits, vocabsize):
        self.numdigits = numdigits
        self.vocabsize = vocabsize
        # create a dataset of vocabsize**numdigits rows and vocabsize columns
        self.data = torch.randint(0, vocabsize, (vocabsize**4, numdigits)).to(device)
        self.data_reversed = torch.flip(self.data, (-1,)).to(device)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def get_data(self):
        return self.data.to(device)

    def get_data_reversed(self):
        return self.data_reversed.to(device)

    def get_num_digits():
        return self.numdigits


dataset = GenerateDataset(numdigits = len_seq, vocabsize = vocab_size)
X = dataset.get_data()
y = dataset.get_data_reversed()


cuda


In [3]:
global ALREADY_PRINT
ALREADY_PRINT = 0

class Embedding(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(Embedding, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        print(self.vocab_size, self.d_model)
        return self.embedding(x)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        # implements the positional encoding function as the size of x
        self.dropout = nn.Dropout(p=dropout)
        max_len = 512
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].requires_grad_(False)
        return self.dropout(x)


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.dk = d_model // num_heads

        self.WO = nn.Linear(d_model, d_model)
        self.WQ = nn.Linear(d_model, d_model)
        self.WK = nn.Linear(d_model, d_model)
        self.WV = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

    def attention(self, Q, K, V, dk, mask=False):
        QKT = torch.matmul(Q, K.transpose(-2, -1))  # output dim: (batch_size, num_heads, n_tokens, n_tokens)
        scaled_dot_product = torch.div(QKT, math.sqrt(dk))

        if mask:
            mask = torch.triu(torch.ones(QKT.size(-2), QKT.size(-1)), diagonal=1).bool().to(device)
            scaled_dot_product = scaled_dot_product.masked_fill(mask, -float('inf'))
            # print("Masked")
            # print(scaled_dot_product)
            # ALREADY_PRINT = 1

        sm = torch.nn.Softmax(dim=-1)
        attention = sm(scaled_dot_product)
        attention = self.dropout(attention)
        return torch.matmul(attention, V)

    def forward(self, Q, K, V, mask=False):
        Q = self.WQ(Q)
        K = self.WK(K)
        V = self.WV(V)

        Q = Q.view(Q.size(0), -1, self.num_heads, self.dk).transpose(1, 2)  # Q: Why -1? A: It's a batch size
        K = K.view(K.size(0), -1, self.num_heads, self.dk).transpose(1, 2)
        V = V.view(V.size(0), -1, self.num_heads, self.dk).transpose(1, 2)

        attention = self.attention(Q, K, V, self.dk, mask)
        attention = attention.transpose(1, 2).contiguous().view(Q.size(0), -1, self.d_model)
        return self.WO(attention)

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.multi_head_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, d_model)
        )

    def forward(self, x, mask):
        x = x + self.dropout1(self.multi_head_attention(x, x, x, mask))
        x = self.norm1(x)

        x = x + self.dropout2(self.ff(x))
        x = self.norm2(x)
        return x

class Decoder(nn.Module):
    def __init__(self, d_model, num_heads, num_layers, dropout=0.1):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, num_heads, dropout) for _ in range(num_layers)])

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return x

class MyTransformerDecoderOnly(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers,  dropout=0.1):
        super(MyTransformerDecoderOnly, self).__init__()
        self.embedding = Embedding(vocab_size, d_model)
        self.transpose_embedding = nn.Linear(d_model, vocab_size)
        self.transpose_embedding.weight = self.embedding.embedding.weight
        self.pos_enc = PositionalEncoding(d_model)
        self.decoder = Decoder(d_model, num_heads, num_layers, dropout)
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, x, mask):
        x = self.embedding(x)
        x = self.pos_enc(x)
        x = self.decoder(x, mask)
        x = self.linear(x)
        # Transpos of emebedding
        # x = self.transpose_embedding(x)

        return x


## Trainning

In [4]:
X[0]
X.size()

torch.Size([10000, 6])

In [5]:
print(y[0])
print(X[0])

tensor([0, 8, 1, 5, 7, 2], device='cuda:0')
tensor([2, 7, 5, 1, 8, 0], device='cuda:0')


In [6]:


train_dataset = TensorDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model = MyTransformerDecoderOnly(vocab_size, d_model, num_heads, num_layers, dropout)
cross_entropy = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0001)

print(device)

model.to(device)
model.train()

for epoch in range(1):
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (X, y) in pbar:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(X, mask=True)
        loss = cross_entropy(y_pred.view(-1, vocab_size), y.view(-1))

        loss.backward()
        optimizer.step()
        pbar.set_description(f'Epoch {epoch+1} - loss: {loss.item():.4f}')




cuda


  0%|          | 0/313 [00:00<?, ?it/s]

10 512


Epoch 1 - loss: 2.2423:   2%|▏         | 6/313 [00:01<00:54,  5.66it/s]

10 512
10 512
10 512
10 512
10 512
10 512
10 512
10 512
10 512


...

10 512
10 512
10 512
10 512
10 512
10 512
10 512


Epoch 1 - loss: 1.1796:  98%|█████████▊| 307/313 [00:08<00:00, 35.02it/s]

10 512
10 512
10 512
10 512
10 512
10 512
10 512


Epoch 1 - loss: 1.1726: 100%|██████████| 313/313 [00:08<00:00, 35.24it/s]

10 512
10 512
10 512
10 512
10 512


In [7]:

model.eval()
X_test = torch.tensor([[9,8,4,6,5,2]]).to(device)
output = model(X_test, mask=True)
predicted = torch.argmax(output, dim=-1)
print(predicted)



10 512
tensor([[5, 5, 4, 4, 8, 9]], device='cuda:0')


In [8]:

model.eval()
X_test = torch.tensor([[1,2,3,4,5,6]]).to(device)
output = model(X_test, mask=True)
predicted = torch.argmax(output, dim=-1)
print(predicted)


10 512
tensor([[5, 5, 3, 3, 2, 1]], device='cuda:0')


# Shakespeare

In [11]:
import re

In [92]:
from torch.utils.data import Dataset

class ShakespeareData(Dataset):
    def __init__(self, words, block_size):
        self.words = words
        self.vocab = sorted(list(set(words)))
        self.idx2word = {i: word for i, word in enumerate(self.vocab)}
        self.word2idx = {word: i for i, word in enumerate(self.vocab)}
        self.block_size = block_size
        self.vocab_size = len(self.vocab)

    def __len__(self):
        return len(self.words) - self.block_size

    def __getitem__(self, idx):
        sequence = self.words[idx:idx + self.block_size + 1]
        encoded_sequence = [self.word2idx[s] for s in sequence]
        x = torch.tensor(encoded_sequence[:-1], dtype=torch.long)
        y = torch.tensor(encoded_sequence[1:], dtype=torch.long)
        return x, y

    def get_vocab(self):
        return self.vocab

    def get_idx2word(self):
        return self.idx2word

    def get_word2idx(self):
        return self.word2idx

    def get_block_size(self):
        return self.block_size

    def get_words(self):
        return self.words

    def get_vocab_size(self):
        return self.vocab_size

    def get_block_size(self):
        return self.block_size


In [93]:
file = "shakespeare.txt"
# file = "/kaggle/input/shakespeare/shakespeare.txt"
with open(file, "r") as f:
    text = f.read()


# text = open(file, 'r').read()
words = re.split(r"\b", text)
block_size = 32
train_dataset = ShakespeareData(words, block_size)
vocab_size = train_dataset.get_vocab_size()

batch_size = 128
train_loader = DataLoader(
    train_dataset, shuffle=True, pin_memory=True, batch_size=batch_size
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)


device: cuda


In [76]:
num_layers = 2
num_heads = 2
d_model = 128
vocab_size = train_dataset.get_vocab_size()

model = MyTransformerDecoderOnly(vocab_size,
                                 d_model,
                                 num_heads,
                                 num_layers,
                                 dropout).to(device).train()

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=6e-4)

max_epochs = 1
for epoch in range(max_epochs):
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for it, (x, y) in pbar:
        x = x.to(device)
        y = y.to(device)
#         print(x.shape)
        optimizer.zero_grad()

        logits = model(x, True)
        loss = loss_fn(logits.view(-1, logits.size(-1)), y.view(-1))
        loss.backward()

        optimizer.step()

        pbar.set_description(f"epoch {epoch} iter {it}: train loss {loss.item():.5f}")

epoch 0 iter 1: train loss 10.27095:   0%|          | 1/15515 [00:00<56:54,  4.54it/s]

32459 128
32459 128
32459 128


epoch 0 iter 5: train loss 9.06332:   0%|          | 5/15515 [00:00<22:23, 11.54it/s]

32459 128
32459 128
32459 128
32459 128


epoch 0 iter 9: train loss 8.15950:   0%|          | 9/15515 [00:00<18:25, 14.03it/s]

...

32459 128
32459 128
32459 128
32459 128


epoch 0 iter 15513: train loss 2.88508: 100%|█████████▉| 15514/15515 [17:12<00:00, 14.87it/s]

32459 128
32459 128


epoch 0 iter 15514: train loss 2.80284: 100%|██████████| 15515/15515 [17:13<00:00, 15.01it/s]

32459 128


In [86]:
block_size

32

In [95]:
model.eval()

# def generate_text(model, dataset, device, seed_text, n_words
# ):
#     block_size = 32
#     word2idx = dataset.get_word2idx()
#     idx2word = dataset.get_idx2word()
#     vocab_size = dataset.get_vocab_size()
#     words = dataset.get_words()
#     model.eval()
#     with torch.no_grad():
#         x = torch.tensor([word2idx[s] for s in seed_text], dtype=torch.long).to(device)
#         for _ in range(n_words):
#             x = x.unsqueeze(0)
#             logits = model(x, mask=True)
#             logits = logits[:, -1, :]
#             probas = F.softmax(logits, dim=-1)
#             next_token = torch.multinomial(probas, num_samples=1)
#             next_token = next_token.item()
#             x = torch.cat((x, torch.tensor([[next_token]], dtype=torch.long).to(device)), dim=1)

#     return [idx2word[i] for i in x[0]]

# generate_text(model, train_dataset, device, "the", 100)



MyTransformerDecoderOnly(
  (embedding): Embedding(
    (embedding): Embedding(32459, 128)
  )
  (transpose_embedding): Linear(in_features=128, out_features=32459, bias=True)
  (pos_enc): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): Decoder(
    (layers): ModuleList(
      (0-1): 2 x DecoderLayer(
        (multi_head_attention): MultiHeadAttention(
          (WO): Linear(in_features=128, out_features=128, bias=True)
          (WQ): Linear(in_features=128, out_features=128, bias=True)
          (WK): Linear(in_features=128, out_features=128, bias=True)
          (WV): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
        (ff): Sequential(
       